# Execise 3.3 (Accelerations)

In [58]:
using NBInclude
using Dates
using LinearAlgebra,Plots

@nbinclude("./SAT_Const.ipynb")
@nbinclude("./SAT_Force.ipynb")
@nbinclude("./SAT_VecMat.ipynb")
@nbinclude("./SAT_RefSys.ipynb")

IERS_UT1_UTC (generic function with 1 method)

- Satellite Orbits (pag 114)

The selection of an appropriate force model for a
specific satellite orbit requires an assessment of the various perturbations acting on
the satellite. To first order the analytical acceleration equations may be evaluated
and the altitude regimes determined, where certain perturbations exceed others.
Determine the altitudes where the acceleration from the Earth’s dominant zonal
gravity term J 20 and sectorial term J 22 equals the acceleration due to the Moon and
Sun. In addition, determine the altitude, where the non-conservative accelerations
due to atmospheric drag and solar radiation pressure balance (C R = 1.3, C D = 2.3).

## Pegasus

In [66]:
function Pegasus( f,
                  LowerBound, UpperBound, Accuracy)
"""
 Purpose:    Root finder using the Pegasus method

 Input
     PegasusFunct  Pointer to the function to be examined
     LowerBound    Lower bound of search interval
     UpperBound    Upper bound of search interval
     Accuracy      Desired accuracy for the root

 Output
     Root          Root found (valid only if Success is true)
     Success       Flag indicating success of the routine

"""

  MaxIterat = 30 
  
  x1 = LowerBound
  x2 = UpperBound
  x3 = 0.0

  f1 = f(x1)
  f2 = f(x2)
  f3 = 0.0

  Iterat = 0
  Success = false
  Root    = x1

  if ( f1 * f2 < 0.0 )

    while (!Success && (Iterat<MaxIterat))
        
      # Approximation of the root by interpolation  
      x3 = x2 - f2/( (f2-f1)/(x2-x1) )
      f3 = f(x3)

      # Replace (x1,f1) and (x2,f2) by new values, such that
      # the root is again within the interval [x1,x2]
      if ( f3 * f2 <= 0.0 ) 
        # Root in [x2,x3]
        x1 = x2
        f1 = f2 
        x2 = x3
        f2 = f3 
      else 
        # Root in [x1,x3]            
        f1 = f1 * f2/(f2+f3)  # Replace (x1,f1) by (x1,f1')
        x2 = x3
        f2 = f3    
      end

      if (abs(f1) < abs(f2))
        Root = x1
      else
        Root = x2
      end

      Success = (abs(x2-x1) <= Accuracy)
      Iterat+=1
    
    end

  end
    
  return(Root,Success)

                    
 end

Pegasus (generic function with 1 method)

## Main

In [3]:
N_JGM3 = 20
GM_Earth = 398600.4415e+9 
R_JGM3 = 6378.1363e3   # // Radius Earth [m]; 
CS_JGM3=zeros(N_JGM3+1,N_JGM3+1)
CS_JGM3!(CS_JGM3,Float64)

GravJGM3=GravModel{Float64}(N_JGM3,GM_Earth,R_JGM3,CS_JGM3)

GravModel{Float64}(20, 3.986004415e14, 6.3781363e6, [1.0 0.0 … 7.160542e-11 2.759192e-9; 0.0 0.0 … -1.008909e-10 3.216826e-10; … ; -2.197334e-8 -3.156695e-9 … -9.155723e-31 -1.237749e-31; 1.203146e-7 3.688524e-9 … -1.878413e-31 4.054696e-32])

In [7]:
fieldnames(typeof(GravJGM3))

(:N, :GM, :R, :CS)

In [57]:
(A_Diff(h1), A_Diff(h2))

lon=0.0,lat=0.0,height=149.9992999999998
lon=0.0,lat=0.0,height=1999.9992999999997


(0.15341603259326977, -5.928000000000001e-6)

In [56]:
function A_Diff(h)
"""
  Function computes the difference of acceleration due to SRP and DRG
    
"""    
  CD     =     2.3      #Spacecraft parameters
  CR     =     1.3
  Mjd_TT = 51269.0      #State epoch

  r = [1.0, 0.0, 0.0]*(GravJGM3.R + h)
  dens = Density_HP(Mjd_TT,r)

  return (0.5*CD*dens*GravJGM3.GM/(GravJGM3.R + h))-(P_Sol*CR)

end

A_Diff (generic function with 1 method)

In [13]:
const r_Moon = 384400.0e+03  # Geocentric Moon distance [m]
const J20_norm =  4.841e-04  # Normalized geopotential coefficients
const J22_norm =  2.812e-06

const h1  =  150.0e+3        # Start of interval [m]
const h2  = 2000.0e+3        # Stop of interval [m]
const eps =  100.0;          # Accuracy [m]  

In [70]:
#Balance of accelerations due to drag and solar radiation pressure
(Root,Success)=Pegasus(A_Diff, h1, h2, eps)
Root/1.e3

750.4211699193136

In [72]:
#Balance of accelerations due to J22 and Moon
r5 = 3.0/2.0 * GravJGM3.GM/GM_Moon * (GravJGM3.R)^2 * r_Moon^3 * J22_norm;
r = r5^(0.2)
(r - GravJGM3.R)/1.e3

8750.075531877797

In [74]:
#Balance of accelerations due to J22 and Sun
r5 = 3.0/2.0 * GravJGM3.GM/GM_Sun * GravJGM3.R^2 * AU^3 * J22_norm;
r = r5^0.2
(r - GravJGM3.R)/1.e3

11297.673064575658

In [75]:
#Balance of accelerations due to J20 and Moon
r5 = 3.0/2.0 * GravJGM3.GM/GM_Moon * (GravJGM3.R)^2 * r_Moon^3 * J20_norm;
r = r5^(0.2)
(r - GravJGM3.R)/1.e3

35983.38520365431

In [76]:
# Balance of accelerations due to J20 and Sun
r5 = 3.0/2.0 * GravJGM3.GM/GM_Sun * GravJGM3.R^2 * AU^3 * J20_norm;
r = r5^0.2
(r - GravJGM3.R)/1.e3

43117.08407456981